The following tutorial will be explained, then expanded on in regards to multi-target tracking: https://pyro.ai/examples/tracking_1d.html

First, to be able to run the tutorial, the following was done to create a runnable environment:
1. Create an environment using anaconda
2. Open a terminal using this environment
3. Install pytorch by running in the terminal _conda installpytorch torchvision -c pytorch_
4. Install pyro by running in the terminal _pip3 install pyro-ppl_

Two things were added to the code base to be able to make it run out of a terminal:
1. The following code was added due to this [stack overflow error recommendation](https://stackoverflow.com/questions/20554074/sklearn-omp-error-15-when-fitting-models):
<pre><code>
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
</code></pre>
2. The code line _pyplot.show()_ at the end of the plot_solution function so the plots would display outside of a Jupyter notebook. 

With the following command:
_python track1D.py_ 
I am able to run the code from the tutorial. Next, I make the points worth noting in what is being run before I adapt and expand the code to different scenarios.

